1. Switch word tagging in both dictionaries and in the output file
2. Using output file, generate 200 GPT-3 outputs from random samples
3. Write F1 score code using dictionaries and GPT-3 output

In [5]:
import os
import openai

import random
import os
import pandas as pd
import numpy as np
import pickle as pkl

#connect to API with Clay's info
# openai.api_key= os.environ['OPENAI_KEY']

In [9]:
!pip install openai


#in create:
  #presence penalty
  #logit bias
  #https://beta.openai.com/docs/api-reference/completions/create
input_filename = "../data/input.txt" # loads this file
output_filename = "../outputs/output.txt" # creates this file
output_map_filename = "../outputs/output_maps.pkl"

# read text file, keep all lines (including those that start with commas)
input = pd.read_csv(input_filename, header=None, sep='\n')
input = input[0].str.split('\s\|\s', expand=True)
sentences = []
tags = []
sentence_maps = []
sentence_maps_space = []

# convert to np array for processing
input = np.asarray(input)

# convert to two separate arrays
for idx, line in enumerate(input):
  if idx % 2 == 0:
    sentence = input[idx][0].lower()
    sentence_space = input[idx][0].lower()
    sentences.append(sentence)

    sentence_map = dict()
    sentence_map_space = dict()
    sentence = sentence.split(' ')
    sentence_space = sentence_space.split(' ')
    for word in sentence_space:
      word = " " + word
    for word in sentence_space:
      if word in sentence_map_space:
        sentence_map_space[word] += 1
      else:
        sentence_map_space[word] = 1
    
    for word in sentence:
      if word in sentence_map:
        sentence_map[word] += 1
      else:
        sentence_map[word] = 1
    sentence_maps.append(sentence_map)
    sentence_maps_space.append(sentence_map_space)
  else:
    tags.append(input[idx][0])

# check if number of lines is equal in both arrays
if np.size(sentences) != np.size(tags):
  raise ValueError('Number of sentence lines not equal to number of tag lines.')

# separate by space, so that we can check that the number of terms is equal in both arrays
sentences_tmp = sentences[:] # copy instead of reference
tags_tmp = tags
for idx, line in enumerate(sentences_tmp):
  sentences_tmp[idx] = sentences_tmp[idx].split()
for idx, tag in enumerate(tags_tmp):
  tags_tmp[idx] = tags_tmp[idx].split()

# check if number of terms is equal in both arrays
if len(sentences_tmp) != len(tags_tmp):
  raise ValueError('Number of terms in "sentences" not equal to number of terms in "tags"')
for i in range(len(sentences_tmp)):
  if len(sentences_tmp[i]) != len(tags_tmp[i]):
    raise ValueError('Unequal number of terms at line (index) ' + str(i))



# create output array and file and save dictionary of correct quantified drug labels for each example
output = []
drugs_maps = []
for line in range(len(sentences_tmp)):
  drugs_current_line = []
  sentence = "Sentence: " + str(sentences[line])
  output.append(sentence)
  for drug_idx, label_list in enumerate(tags[line]):
    if tags[line][drug_idx] == "B":
      temporary_drug_term = sentences_tmp[line][drug_idx]
      drugs_current_line.append(sentences_tmp[line][drug_idx])
    if tags[line][drug_idx] == "I":
      while drugs_current_line:
        popped = drugs_current_line.pop()
      temporary_drug_term = sentences_tmp[line][drug_idx]
      new_word = popped + temporary_drug_term
      drugs_current_line.append(new_word)
      # print(drugs_current_line)
      drugs_current_line.append(sentences_tmp[line][drug_idx])
  drugs = "Drugs: " + ", ".join(str(x).lower() for x in drugs_current_line)
  output.append(drugs)
  print(drugs_current_line)
  #create and save dictionary of drug names
  drugs_map = dict()
  for drug in drugs_current_line:
    if drug in drugs_map:
      drugs_map[drug] += 1
    else:
      drugs_map[drug] = 1
  drugs_maps.append(drugs_map)

with open(output_filename, mode='wt') as output_file:
    output_file.write('\n'.join(output)) # give it a minute to save

['naloxone', 'clonidine']
['clonidine']
['nalozone']
['alpha-methyldopa', 'methyldopa', 'naloxone']
['naloxone']
['clonidine']
['[3h]-naloxone', 'naloxone', 'naloxone']
[]
['clonidine3h]-dihydroergocryptine', 'dihydroergocryptine']
[]
['naloxone', 'clonidine', 'clonidine']
['alpha-methyldopa', 'methyldopa']
['lidocaine']
['lidocaine']
[]
[]
['lidocaine']
['suxamethonium']
[]
['suxamethoniumchloride', 'chloride']
['sch']
[]
['sch']
[]
[]
[]
[]
[]
[]
[]
['galanthaminehydrobromide', 'hydrobromide', 'scopolamine', 'hyoscine']
['galanthaminehydrobromide', 'hydrobromide', 'scopolamine', 'hyoscine']
['physostigmine']
['scopolamine']
['lithium']
['lithium']
[]
['lithium']
[]
['lithium']
['lithium', 'lithium', 'lithium']
['lithium']
['lithium', 'li']
['lithium']
['creatinine', 'lithium']
[]
['li']
['li']
[]
[]
[]
['fusidicacid', 'acid', 'cyclosporin']
['cyclosporin']
['fusidicacid', 'acid']
[]
[]
['fusidicacid', 'acid']
[]
[]
['fusidicacid', 'acid']
[]
[]
[]
[]
['fusidicacid', 'acid']
['fusidic

['deferoxamine']
['deferoxamine']
[]
[]
['deferoxamine']
[]
[]
[]
['flurbiprofen']
['flurbiprofen']
[]
[]
[]
[]
[]
[]
[]
['mipafox']
['mipafox']
["n,n'-diisopropylphosphorodiamidofluoridate", 'diisopropylphosphorodiamidofluoridate', 'organophosphate']
['mipafox']
[]
[]
[]
[]
[]
['mipafox']
[]
['mipafox']
['phenylpropanolamine']
['phenylpropanolamine', 'ppa', 'amphetamine']
[]
['ppa']
['azathioprine']
['azathioprine']
[]
[]
['azathioprine']
['lithium']
[]
['lithium']
[]
[]
['lithium', 'lithium']
[]
[]
['isoproterenol']
[]
[]
[]
['isoproterenols']
[]
[]
[]
['polyethyleneglycol400', '400', 'adriamycin']
['polyethyleneglycol400400', '400', 'adriamycin']
['adr']
['peg400', '400', 'adr']
['adr']
['adr']
['bcnu']
['bcnu,3-bis-(2-chloroethyl)-1-nitrosourea', 'nitrosourea']
[]
['bcnu']
[]
[]
[]
[]
[]
[]
[]
['bcnu']
[]
['bcnu']
[]
['ethanol']
['noradrenaline']
['sodiumchloride', 'chloride', 'noradrenaline']
[]
['noradrenaline']
['noradrenaline', 'noradrenaline', 'noradrenaline', 'noradrenaline']

['bilirubin']
['pge1']
['levodopa']
['levodopa']
['dopamine']
['levodopa']
[]
[]
['dopamine', 'glutamate']
['levodopa']
[]
['pilocarpine']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['salvianolicacida', 'a', 'isoproterenol']
['salvianolicacida', 'a', 'isoproterenol']
[]
[]
[]
['isoproterenol', 'lactate', 'aspartate', 'creatine', 'malondialdehyde', 'superoxide']
['glutathione']
[]
[]
['adp', 'isoproterenol']
['salvianolicacida', 'a', 'isoproterenol']
['salvianolicacida', 'a', 'isoproterenol']
['salvianolicacida', 'a', 'isoproterenol']
['n-(2-propylpentanoyl)ureaacid', 'acid', 'pilocarpine']
['aminoacid', 'acid', 'glutamate', 'aspartate', 'glycine']
['gaba-(2-propylpentanoyl)ureaacid', 'acid', 'vpa']
['vpu', 'vpa', 'pilocarpine']
['vpa']
['pilocarpine', 'glutamate', 'aspartate']
['glycine', 'gaba']
['vpu', 'vpa', 'pilocarpine']
['glutamate', 'aspartate']
['gaba', 'glycine', 'glutamate', 'aspartate']
['vpuacid', 'acid']
['glutamate', 'aspartate']
['vpa', 'vpu', 'pilocarpine', 'glutamate', 'aspartate']

['nicotine', 'da']
['nicotine', 'nicotine', 'da']
['da']
['hydrocortisone']
['hydrocortisone']
['hydrocortisone']
['norepinephrine']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['norepinephrine', 'hydrocortisone']
[]
[]
['hydrocortisone']
[]
['norepinephrine']
['norepinephrine', 'norepinephrine']
['hydrocortisone']
[]
['suprofen']
['suprofen']
['suprofen']
['uricacid', 'acid']
['sodium', 'oxygen', 'suprofen']
[]
['suprofenacid', 'acid']
['suprofenacid', 'acid']
['uricacid', 'acid', 'suprofen']
['uricacid', 'acid']
['suprofenacid', 'acid']
['cocaine']
[]
['cocaine']
[]
['cocaine']
[]
['cocaine']
[]
[]
[]
[]
[]
['cocaine']
[]
['deoxycholicacidestradiol', 'estradiol']
['deoxycholicacidestradiol', 'estradiol']
['deoxycholicacidacidestradiol', 'estradiol']
['sodiumdeoxycholateacid', 'acid']
['ethinylestradiol', 'estradiol']
['ethinylestradiolacidacid', 'acid']
['ethinylestradiolacidacid', 'acid']
[]
['deoxycholicacid', 'acid']
['deoxycholicacidacid', 'acid']
['ethinylestradiolacid', 'acid']
['ouabain', 'oxy

['steroids']
['pilocarpine']
[]
[]
[]
['steroids', 'benzodiazepine', 'clonazepam', 'pilocarpine', 'steroids']
['clonazepam', 'steroids']
[]
['steroids']
['kainicacid', 'acid']
['steroidacid', 'acid']
[]
['steroids', 'nmda']
['nmda']
['steroidsacid', 'acid']
[]
['n-butyl-deoxynojirimycin-48334', '48334', 'zidovudine']
[]
['n-butyl-deoxynojirimycin', 'deoxynojirimycin']
['sc-48334', '48334', 'zidovudine', 'zidovudine']
['zidovudine-48334', '48334']
['zidovudine', 'zidovudine']
['zidovudine']
['zidovudine']
[]
['sc-48334', '48334']
[]
['zidovudine']
[]
['zidovudine', 'zidovudine']
[]
['zidovudine']
[]
[]
[]
[]
['n10-propargyl-5,8-dideazafolicacid3717', '3717']
['cb3717-propargyl-5,8-dideazafolicacid', 'acid']
[]
['cb3717', '3717']
['cb3717', '3717']
['cb3717', '3717']
[]
['cb37173717', '3717']
['cb3717', '3717']
[]
['cb3717', '3717']
['cb3717', '3717']
[]
[]
[]
['cb3717', '3717']
['ethopropazine', 'benztropine']
['ethopropazineenanthate', 'enanthate']
['ethopropazine', 'benztropine', 'pro

In [10]:
count = 0
print(drugs_maps)
for entry in drugs_maps:
  for key in entry:
    if entry[key] > 1:
      count += 1
      break
print(count/len(drugs_maps))
print(len(drugs_maps))
print(len(sentence_maps))

[{'naloxone': 1, 'clonidine': 1}, {'clonidine': 1}, {'nalozone': 1}, {'alpha-methyldopa': 1, 'methyldopa': 1, 'naloxone': 1}, {'naloxone': 1}, {'clonidine': 1}, {'[3h]-naloxone': 1, 'naloxone': 2}, {}, {'clonidine3h]-dihydroergocryptine': 1, 'dihydroergocryptine': 1}, {}, {'naloxone': 1, 'clonidine': 2}, {'alpha-methyldopa': 1, 'methyldopa': 1}, {'lidocaine': 1}, {'lidocaine': 1}, {}, {}, {'lidocaine': 1}, {'suxamethonium': 1}, {}, {'suxamethoniumchloride': 1, 'chloride': 1}, {'sch': 1}, {}, {'sch': 1}, {}, {}, {}, {}, {}, {}, {}, {'galanthaminehydrobromide': 1, 'hydrobromide': 1, 'scopolamine': 1, 'hyoscine': 1}, {'galanthaminehydrobromide': 1, 'hydrobromide': 1, 'scopolamine': 1, 'hyoscine': 1}, {'physostigmine': 1}, {'scopolamine': 1}, {'lithium': 1}, {'lithium': 1}, {}, {'lithium': 1}, {}, {'lithium': 1}, {'lithium': 3}, {'lithium': 1}, {'lithium': 1, 'li': 1}, {'lithium': 1}, {'creatinine': 1, 'lithium': 1}, {}, {'li': 1}, {'li': 1}, {}, {}, {}, {'fusidicacid': 1, 'acid': 1, 'cycl

In [11]:
# write maps to .pkl
maps = (sentence_maps, drugs_maps)
print(maps[1][2])
pkl.dump(maps, open(output_map_filename, "wb"))

#test retrieving from .pkl
maps_temp = pkl.load(open(output_map_filename, "rb"))
print(maps_temp[1][2])

{'nalozone': 1}
{'nalozone': 1}


#NER with GPT-3

In [12]:
sentence_maps_space[1]
# sentence_maps[1]

{'in': 2,
 'unanesthetized': 1,
 ',': 3,
 'spontaneously': 1,
 'hypertensive': 1,
 'rats': 1,
 'the': 1,
 'decrease': 1,
 'blood': 1,
 'pressure': 1,
 'and': 1,
 'heart': 1,
 'rate': 1,
 'produced': 1,
 'by': 1,
 'intravenous': 1,
 'clonidine': 1,
 '5': 1,
 'to': 1,
 '20': 1,
 'micrograms': 1,
 '/': 1,
 'kg': 1,
 'was': 1,
 'inhibited': 1,
 'or': 1,
 'reversed': 1}

In [13]:
# !pip install transformers
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# print(tokenizer("nicotine")['input_ids'])
# tokenizer.convert_ids_to_tokens([220])

In [15]:
#generate list of prompt examples and produce prompt string
num_test_samples = 500
num_prompt_samples = 5

# random
prompt_indices = random.sample(range(len(sentence_maps)),num_test_samples + num_prompt_samples)

# non-random 
# prompt_indices = list(range(num_test_samples + num_prompt_samples))


sentence_indices = []
for i in range(len(prompt_indices)):
  sentence_indices.append(prompt_indices[i])
  prompt_indices[i] *= 2
test_sentence_indices = sentence_indices[num_prompt_samples:]
prompts = []
for i in range(num_test_samples):
  prompts.append("")

lastval = prompt_indices[-1]

test_sentence_token_sets = []

for test_prompt_idx in range(len(prompts)):

  for ex_idx in prompt_indices[:num_prompt_samples]:

    #add sample sentences and outputs to prompt string
    prompts[test_prompt_idx] += output[ex_idx] + '\n' + output[ex_idx + 1] + '\n\n'

  #add test example to end of prompt
  prompts[test_prompt_idx] += output[prompt_indices[num_prompt_samples + test_prompt_idx]] + '\n' + 'Drugs:'

  #add set of tokens in prompt to list of sets
  test_sentence_token_set = set()
  for key in sentence_maps_space[int(prompt_indices[num_prompt_samples + test_prompt_idx]/2)]:
    test_sentence_token_set.add(key)
  test_sentence_token_sets.append(test_sentence_token_set)

  # print("tokens in test set", test_sentence_token_set)

In [16]:
prompts

['Sentence: laboratory measurement of pre - procedure serum calcium levels in selected donors may identify cases requiring heightened vigilance .\nDrugs: calcium\n\nSentence: forty - eight hours after admission , the children appeared to be at their baseline and were discharged home .\nDrugs: \n\nSentence: , 2 ) ntg immediately after learning , 3 ) ntg 3 h after learning , 4 ) ntg and nimo , 5 ) vehicle , and 6 )\nDrugs: ntg, ntg, ntg, nimo\n\nSentence: the ph adjustment of standard lidocaine can be accomplished easily in the catheterization laboratory before injection and results in a reduction of the pain occurring during the infiltration of tissues\nDrugs: lidocaine\n\nSentence: a restrictive pattern of valvular regurgitation , suggestive of the role of pergolide , was observed in 12 / 30 ( 40 % ) patients including two with heart failure\nDrugs: pergolide\n\nSentence: we report a woman with coronary artery disease who developed a markedly prolonged qt interval and torsades de point

In [ ]:

#GENERATE PREDICTIONS
predictions_maps = []
counter = 0
for prompt in prompts:
  predictions_map = dict()
  #constrain potential output to tokens in test sentence using logit bias
  logit_biases = {}
  for word in test_sentence_token_sets[counter]:
    print(word)
    tokens = tokenizer(word)['input_ids']
    for token in tokens:
      print(token)
      if token not in logit_biases:
        logit_biases[token] = .1
  print(logit_biases)
  #use API to generate completion
  sample = openai.Completion.create(engine="ada",
                          prompt=prompt,
                          max_tokens=10,
                          temperature=0,
                          # logit_bias = logit_biases,
                          presence_penalty=-0.001,
                          stop=["\n", "<|endoftext|>"])
  prediction = sample['choices'][0]['text']
  #print(prediction)
  prediction_tokens = prediction.replace(",","").split()
  for token in prediction_tokens:
    if token.lower() in test_sentence_token_sets[counter]: #ensures token is in test sentence
      # print("YES")
      if token.lower() in predictions_map:
        # print(token.lower())
        # print(predictions_map)
        predictions_map[token.lower()] += 1
      else:
        predictions_map[token.lower()] = 1
  predictions_maps.append(predictions_map)
  counter += 1


# print(prompts[test_prompt_idx])
# print(test_prompt_idx) #0

# print("Prompt: " + str(sentences_tmp[test_sentence_indices[0]])) # only works for one test sample
# print("Before removing extraneous: " + str(predictions_maps))

only_in_prompt = {}

# for idx in 
# for key, value in predictions_maps[0].items():
#   if key in sentences_tmp[test_sentence_indices[0]]:
#     only_in_prompt[key] = value

# print("After removing extraneous: " + str(only_in_prompt))


# #TEST
# for item in predictions_maps:
#   print(item,"\n\n\n")
# print(prompts[0])

In [ ]:
print(predictions_maps)

[{'clonidine': 1}, {'naloxone': 1}, {}, {'clonidine': 1}, {}, {'clonidine': 1, 'naloxone': 1}, {}, {'lidocaine': 1}, {'lidocaine': 1}, {}, {}, {'lidocaine': 1}, {'suxamethonium': 1}, {}, {'suxamethonium': 1, 'chloride': 1}, {'(': 1, 'sch': 1, ')': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'scopolamine': 1, 'galanthamine': 1}, {'scopolamine': 1, 'galanthamine': 1}, {'physostigmine': 1}, {'scopolamine': 1}, {'lithium': 1}, {'lithium': 1}, {}, {'lithium': 1}, {}, {'lithium': 1}, {'lithium': 1}, {'lithium': 1}, {'lithium': 1}, {'lithium': 1}, {'lithium': 1}, {}, {'li': 1}, {'li': 1}, {}, {}, {}, {'fusidic': 1, 'acid': 1, 'cyclosporin': 1}, {'fusidic': 1, 'acid': 1}, {'fusidic': 1, 'acid': 1}, {}]


In [ ]:
print(tp + fp)

30


In [ ]:
sample = openai.Completion.create(engine="davinci",
                          prompt=prompt,
                          max_tokens=10,
                          logp
                          temperature=0,
                          presence_penalty=-0.001,
                          stop=["\n", "<|endoftext|>"])
prediction = sample['choices'][0]['text']

one


In [ ]:
#COMPUTE CoNLL F1 SCORE

tp = 0
fp = 0
tn = 0
fn = 0

sentence_total=0

print(prompts[0])
#get indices of sentences to get entries from dict
# sentence_indices = sentence_indices[num_prompt_samples:] #ONLY RUN ONCE EACH TIME!
counter = 0
for sentence_idx in test_sentence_indices:
  print(output[prompt_indices[num_prompt_samples + counter]])
  print("Words in sentence:",sentence_maps[sentence_idx])
  print("True labels:",drugs_maps[sentence_idx])
  print("Predicted labels:",predictions_maps[counter],'\n')


  #increment tp and fn
  for key in drugs_maps[sentence_idx]:
    if key in predictions_maps[counter]:
      tp += min(drugs_maps[sentence_idx][key], predictions_maps[counter][key])
    else: #key not in prediction
      fn += drugs_maps[sentence_idx][key]

  #increment fp and fn
  for key in predictions_maps[counter]:
    if key not in drugs_maps[sentence_idx]:
      fp += predictions_maps[counter][key]
    elif key in drugs_maps[sentence_idx] and predictions_maps[counter][key] > drugs_maps[sentence_idx][key]:
      fp += predictions_maps[counter][key] - drugs_maps[sentence_idx][key]
    elif key in drugs_maps[sentence_idx] and predictions_maps[counter][key] < drugs_maps[sentence_idx][key]:
      fn += drugs_maps[sentence_idx][key] - predictions_maps[counter][key]


  #increment tn
  for key in sentence_maps[sentence_idx]:
    sentence_total += sentence_maps[sentence_idx][key]
    if key not in predictions_maps[counter] and key not in drugs_maps[counter]:
      tn += sentence_maps[sentence_idx][key]
    else: #key in prediction
      if key in predictions_maps[counter] and sentence_maps[sentence_idx][key] > predictions_maps[counter][key] and key not in drugs_maps[counter]:
        tn += sentence_maps[sentence_idx][key] - predictions_maps[counter][key]

  counter += 1

precision = tp / (tp + fp)
recall = tp / (tp + fn)

f1 = 2 * (precision * recall) / (precision + recall)
print(f1)
print("precision: {}  recall: {}  tp: {}  fp: {}  tn: {}  fn: {}".format(precision, recall, tp, fp, tn, fn))

Sentence: cells / mm3 at week 8 and 6 . 8 cells / mm3 and - 45 . 1 cells / mm3 at week 16 , respectively .
Drugs: 

Sentence: .
Drugs: 

Sentence: fenoldopam mesylate is a specific da1 receptor agonist that lowers blood pressure by vasodilatation .
Drugs: fenoldopammesylate, mesylate

Sentence: kf17837 is a novel selective adenosine a2a receptor antagonist .
Drugs: kf17837, adenosine

Sentence: or attention / executive functioning ( p = 0 . 59 ) .
Drugs: 

Sentence: hypersensitivity to various auditory , tactile , and visual stimulation was present and shifts in the brainstem ambient power spectral frequency occurred in response to tactile stimulation .
Drugs:
Sentence: hypersensitivity to various auditory , tactile , and visual stimulation was present and shifts in the brainstem ambient power spectral frequency occurred in response to tactile stimulation .
Words in sentence: {'hypersensitivity': 1, 'to': 2, 'various': 1, 'auditory': 1, ',': 2, 'tactile': 2, 'and': 2, 'visual': 1, 'sti

In [ ]:
predictions_maps[0]

{'lithium': 1}

In [ ]:
print(output[0])

Sentence: famotidine - associated delirium .


In [ ]:
## Old code, prints Sentences + Drugs ##

import pandas as pd
import numpy as np

input_filename = "input.txt" # loads this file
output_filename = "output.txt" # creates this file

# read text file, keep all lines (including those that start with commas)
input = pd.read_csv(input_filename, header=None, sep='\n')
input = input[0].str.split('\s\|\s', expand=True)

sentences = []
tags = []

# convert to np array for processing
input = np.asarray(input)

# convert to two separate arrays
for idx, line in enumerate(input):
  if idx % 2 == 0:
    sentences.append(input[idx][0])
  else:
    tags.append(input[idx][0])


# check if number of lines is equal in both arrays
if np.size(sentences) != np.size(tags):
  raise ValueError('Number of sentence lines not equal to number of tag lines.')

# separate by space, so that we can check that the number of terms is equal in both arrays
sentences_tmp = sentences[:] # copy instead of reference
tags_tmp = tags
for idx, line in enumerate(sentences_tmp):
  sentences_tmp[idx] = sentences_tmp[idx].split()
for idx, tag in enumerate(tags_tmp):
  tags_tmp[idx] = tags_tmp[idx].split()


# check if number of terms is equal in both arrays
if len(sentences_tmp) != len(tags_tmp):
  raise ValueError('Number of terms in "sentences" not equal to number of terms in "tags"')
for i in range(len(sentences_tmp)):
  if len(sentences_tmp[i]) != len(tags_tmp[i]):
    raise ValueError('Unequal number of terms at line (index) ' + str(i))



# create output array and file
output = []
for line in range(len(sentences_tmp)):
  drugs_current_line = []
  sentence = "Sentence: " + str(sentences[line])
  output.append(sentence)
  for idx, label_list in enumerate(tags[line]):
    if tags[line][idx] == "B":
      drug_idx = idx
      drugs_current_line.append(sentences_tmp[line][drug_idx])
  drugs = "Drugs: " + ", ".join(str(x).lower() for x in drugs_current_line)
  output.append(drugs)

with open(output_filename, mode='wt') as output_file:
    output_file.write('\n'.join(output)) # give it a minute to save